# Módulo 3: Ejercicio de evaluación final - ELISABET AGUILÓ THIÓ

## Pre-Fase: Comprensión de los Datos

Los datos proporcionados consisten en dos datasets que, en conjunto, describen el comportamiento de los clientes dentro de un programa de lealtad de una aerolínea.

- Customer Flight Analysis.csv muestra información sobre la actividad de vuelo de los clientes (número de vuelos reservados, distancia volada, puntos acumulados, acompañantes, etc.).

- Customer Loyalty History.csv contiene el perfil detallado de los clientes (ubicación, nivel educativo, ingresos, estado civil, etc.).


## Fase 1: Exploración y Limpieza

## 1.1. Librerías 

In [68]:
# ------------------------------
# Manipulación de datos
# ------------------------------
import pandas as pd        # pandas → trabajar con tablas (DataFrames)
import numpy as np         # numpy → operaciones numéricas y manejo de nulos (np.nan)
import os                  # os → interacción con el sistema operativo (verificar existencia de archivos)

# ------------------------------
# Configuración del entorno
# ------------------------------
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en la salida
pd.set_option('display.max_rows', 100)      # Limitar filas mostradas a 100 (para evitar salidas excesivas)


## 1.2. Metodología

- Antes de juntar los datos, revisamos y limpiamos cada CSV por separado.
- Cada archivo puede tener datos incompletos, errores, duplicados, inconsistencias y proceder de distintas fuentes que causen problemas al combinarlos.
- Limpiarlos individualmente garantiza que los datos sean correctos para el análisis.


## 1.3 CSV A – “Customer Flight Activity”

In [69]:
# Definimos la ruta donde está el archivo (carpeta "data" + nombre del archivo)
ruta = "data/Customer Flight Activity.csv"

# Verificamos si el archivo existe en esa ruta
if os.path.exists(ruta):
    # Si existe, lo leemos y lo convertimos en un DataFrame
    df_flight = pd.read_csv(ruta)
    print("✅ Archivo cargado correctamente")
else:
    # Si no existe, mostramos un mensaje de error
    print("⚠️ Archivo no encontrado")
    df_flight = None


✅ Archivo cargado correctamente


In [70]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔍 EXPLORACIÓN INICIAL - Customer Flight Activity.csv
# ═══════════════════════════════════════════════════════════════════════════════

# 1️⃣ ¿QUÉ ESTRUCTURA Y TAMAÑO TIENE?
# Cuando usas shape, lo que estás obteniendo es una tupla con dos valores (filas, columnas): 
print(f"Filas: {df_flight.shape[0]:,} | Columnas: {df_flight.shape[1]}")
# df_flight.columns ya es suficiente para ver las columnas, pero nos muestra un Index
# si queremos una lista de las columnas, podemos convertirlo a lista con tolist()
print("Columnas:", df_flight.columns.tolist())

# Anotaciones: 
# COLUMNAS = "¿QUÉ PUEDO MEDIR?"
# 10 columnas = 10 cosas distintas (número de vuelos reservados, distancia volada, puntos acumulados, acompañantes, etc.)
# FILAS = "¿CUÁNTOS CASOS TENGO?"
# Más de 400K filas = grande (por encima de 100K) → Esto significa que hay muchos registros (muchos casos), los resultados reflejaran mejor la realidad que teniendo pocos casos.


Filas: 405,624 | Columnas: 10
Columnas: ['Loyalty Number', 'Year', 'Month', 'Flights Booked', 'Flights with Companions', 'Total Flights', 'Distance', 'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed']


In [ ]:
# 2️⃣ ¿QUÉ CONTIENE? ¿TIPOS DE DATOS Y NULOS?
df_flight.info()
# podemos profundizar más que con el shape: además de ver las filas y columnas,
# vemos tipos de datos y cuántos valores nulos tiene cada columna

# Anotaciones:
# - El dataset tiene 405,624 registros y 10 columnas.
# - La mayoría de los datos son números enteros (sin decimales).
# - Solo la columna 'Points Accumulated' tiene decimales.
# - No hay columnas categóricas, es decir: no hay columnas de tipo object. 
# - En la fase de limpieza se valorará convertir Year y Month a categóricas,
#   ya que son etiquetas/identificadores y no se operan matemáticamente.
# - También se valorará convertir Loyalty Number a string,
#   porque tiene muchos valores únicos y no conviene tratarlo como categoría.
# - No hay valores faltantes ( valores nulos) en ninguna columna (405624 non-null).


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


In [72]:
# 3️⃣ ¿CÓMO SE VEN LOS DATOS?
# display() se usa para mostrar tablas de forma clara en un notebook (mejor que print)

display(df_flight.head())      
display(df_flight.tail())      
display(df_flight.sample(5, random_state=42))  # Muestra filas al azar. random_state=42 → siempre las mismas

#Anotaciones
# Primeras filas
# - Verifica: que los datos se cargaron bien y muestra las columnas
# - Cada fila es la actividad mensual de un cliente (Year + Month).
# - Los nombres de las columnas pueden generar errores. En fase de limpieza, eliminar espacios y mayúsculas para facilitar análisis posteriores. 
# - Total Flights podría ser la suma de Flights Booked + Flights with Companions. En la fase de limpieza, comprobar.

# - Los ceros en la columna Points redeemed ( canjeo de puntos) pueden indicar: 
# 1. Vuela → Acumula puntos (SIEMPRE, Points accumulated)
# 2. Canjea → Gasta puntos (NO SE OBSERVA, el cliente puede no tener suficientes puntos para canjearlos o por desconocimiento).

# Últimas filas
# - Detecta: filas finales que no son datos reales (suma de totales o notas que no son registros de clientes)
# - En nuestro caso, las últimas filas muestran solo datos reales de clientes.

# Muestra aleatoria
# - Revisar filas al azar ayuda a detectar errores no visibles al inicio o final.
# - En la muestra no se observan errores evidentes.


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
405619,999902,2018,12,0,0,0,0,0.0,0,0
405620,999911,2018,12,0,0,0,0,0.0,0,0
405621,999940,2018,12,3,0,3,1233,123.0,0,0
405622,999982,2018,12,0,0,0,0,0.0,0,0
405623,999986,2018,12,0,0,0,0,0.0,0,0


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
182259,802535,2017,11,0,0,0,0,0.0,0,0
78391,674643,2017,5,0,0,0,0,0.0,0,0
24345,496371,2017,2,4,4,8,1512,151.0,0,0
297818,659419,2018,6,0,0,0,0,0.0,0,0
203891,158560,2018,1,3,0,3,1911,191.0,0,0


In [ ]:
# 4️⃣ ¿QUÉ NÚMEROS HAY?
# describe().T muestra estadísticas básicas y las filas quedan como columnas (más fácil de leer)

display(df_flight.describe().T)  

# Anotaciones

# Integridad ( count):
# - Todas las columnas tienen 405,624 registros (count). 
# - Confirmamos que no hay valores nulos; los datos están completos.

# Rangos y Coherencia (min y max):
# - Años: Solo 2017 y 2018.
# - Meses: Del 1 al 12.

# Relación entre media y mediana (50%), columna Total Flights: 
# - Media: 5.1. 
# - Mediana: 1.0. 
# Nos tenemos que guiar por la mediana para entender el comportamiento típico.
# El comportamiento típico es volar 1 vez al mes.
# Pero la media es 5.1, no es igual a la mediana, lo que indica que hay valores atípicos (outliers) que elevan la media.
# Es decir, la mayoría de los clientes vuelan 1 vez al mes, pero algunos "viajeros frecuentes" (outliers) vuelan mucho más (32 máximo).

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,405624.0,550037.873084,258935.286969,100018.0,326961.00,550834.0,772194.00,999986.0
Year,405624.0,2017.500000,0.500001,2017.0,2017.00,2017.5,2018.00,2018.0
Month,405624.0,6.500000,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,405624.0,4.115052,5.225518,0.0,0.00,1.0,8.00,21.0
Flights with Companions,405624.0,1.031805,2.076869,0.0,0.00,0.0,1.00,11.0
Total Flights,405624.0,5.146858,6.521227,0.0,0.00,1.0,10.00,32.0
Distance,405624.0,1208.880059,1433.155320,0.0,0.00,488.0,2336.00,6293.0
Points Accumulated,405624.0,123.692721,146.599831,0.0,0.00,50.0,239.00,676.5
Points Redeemed,405624.0,30.696872,125.486049,0.0,0.00,0.0,0.00,876.0
Dollar Cost Points Redeemed,405624.0,2.484503,10.150038,0.0,0.00,0.0,0.00,71.0


In [74]:
# 5️⃣ ¿Hay errores en los datos?
print("🔴 NULOS POR COLUMNA:", df_flight.isnull().sum()) 
print("🔴 FILAS DUPLICADAS:", df_flight.duplicated().sum()) 
print("🔴 % DUPLICADOS:", df_flight.duplicated().mean() * 100) 
print("🔴 VALORES ÚNICOS:\n", df_flight.nunique()) 

# Anotaciones 

# INTEGRIDAD: 
# No hay valores nulos → Dataset completo, listo para análisis

# DUPLICADOS: 
# 1,864 filas repetidas (0.46%) → Riesgo de distorsionar los datos
# Acción en fase posterior de limpieza: Eliminar con drop_duplicates()

# nunique() - VALORES ÚNICOS:
# Revela variedad real por columna
# - Loyalty Number: 16,737 clientes únicos (lógico: múltiples meses/cliente)
# - Month: 12 valores (correcto: solo 12 meses posibles)
# - Year: 2 años (correcto: 2017-2018)

🔴 NULOS POR COLUMNA: Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64
🔴 FILAS DUPLICADAS: 1864
🔴 % DUPLICADOS: 0.4595388833007909
🔴 VALORES ÚNICOS:
 Loyalty Number                 16737
Year                               2
Month                             12
Flights Booked                    22
Flights with Companions           12
Total Flights                     33
Distance                        4746
Points Accumulated              1549
Points Redeemed                  587
Dollar Cost Points Redeemed       49
dtype: int64


In [97]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔍 LIMPIEZA - Customer Flight Activity.csv
# ═══════════════════════════════════════════════════════════════════════════════

# 1) Cambiar el tipo de variable
df_flight['loyalty_number'] = df_flight['loyalty_number'].astype('string')
df_flight['year'] = df_flight['year'].astype('category')
df_flight['month'] = df_flight['month'].astype('category')
print("Tipo de variables:")
print(df_flight[['loyalty_number','year','month']].dtypes)

# 2) Limpiar nombres de columnas (snake_case)
df_flight.columns = df_flight.columns.str.lower().str.replace(' ', '_')
print( "Las columnas tienen los nombres correctos", df_flight.columns.tolist())

# 3) Total Flights podría ser la suma de Flights Booked + Flights with Companions.
print("Verificamos si la columna total_flights es producto de la suma:", 
      (df_flight['total_flights'] == df_flight['flights_booked'] + df_flight['flights_with_companions']).all())

# 4) Eliminar duplicados
df_flight = df_flight.drop_duplicates()
# 4) Verificar duplicados 
print("El número de duplicados después de la limpieza:", df_flight.duplicated().sum())


Tipo de variables:
loyalty_number    string[python]
year                    category
month                   category
dtype: object
Las columnas tienen los nombres correctos ['loyalty_number', 'year', 'month', 'flights_booked', 'flights_with_companions', 'total_flights', 'distance', 'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed']
Verificamos si la columna total_flights es producto de la suma: True
El número de duplicados después de la limpieza: 0
